## NAME : JAYNIL GAGLANI
## ROLL NO : 13
## DATA WAREHOUSING AND MINING
## EXPERIMENT NO 6

**BigMart Sales Prediction practice problem**

We have train (8523) and test (5681) data set, train data set has both input and output variable(s). We need to predict the sales for test data set.


* Item_Identifier: Unique product ID

* Item_Weight: Weight of product

* Item_Fat_Content: Whether the product is low fat or not

* Item_Visibility: The % of total display area of all products in a store allocated to the particular product

* Item_Type: The category to which the product belongs

* Item_MRP: Maximum Retail Price (list price) of the product

* Outlet_Identifier: Unique store ID

* Outlet_Establishment_Year: The year in which store was established

* Outlet_Size: The size of the store in terms of ground area covered

* Outlet_Location_Type: The type of city in which the store is located

* Outlet_Type: Whether the outlet is just a grocery store or some sort of supermarket

* Item_Outlet_Sales: Sales of the product in the particulat store. This is the outcome variable to be predicted.

**Load Libraries**

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

In [ ]:

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))


**Data Mining**

In [ ]:
train = pd.read_csv('../input/Train.csv')
test = pd.read_csv('../input/Test.csv')

**Exploratory Data Analysis**

In [ ]:
train['source'] = 'train'
test['source'] = 'test'
test['Item_Outlet_Sales'] = 0
data = pd.concat([train, test], sort = False)
print(train.shape, test.shape, data.shape)

In [ ]:
data['Item_Outlet_Sales'].describe()

In [ ]:
sns.distplot(data['Item_Outlet_Sales'])

1. Deviate from the normal distribution.
1. Have appreciable positive skewness.
1. Show peakedness.

In [ ]:
print('Skewness: %f' % data['Item_Outlet_Sales'].skew())
print('Kurtsis: %f' %data['Item_Outlet_Sales'].kurt())

In [ ]:
categorial_features = data.select_dtypes(include=[np.object])
categorial_features.head(2)

In [ ]:
numerical_features = data.select_dtypes(include=[np.number])
numerical_features.head(2)

**Finding Missing values**

**Data Cleaning and Imputing Missing Values**

We found two variables with missing values – Item_Weight and Outlet_Size. Lets impute the former by the average weight of the particular item. This can be done as:

In [ ]:
item_avg_weight = data.pivot_table(values='Item_Weight', index='Item_Identifier')

missing_values = data['Item_Weight'].isnull()
print('Missing values: %d' %sum(missing_values))

data.loc[missing_values,'Item_Weight']  = data.loc[missing_values,'Item_Identifier'].apply(lambda x: item_avg_weight.at[x,'Item_Weight'])
print('Missing values after immputation %d' %sum(data['Item_Weight'].isnull()))

This confirms that the column has no missing values now. Lets impute Outlet_Size with the mode of the Outlet_Size for the particular type of outlet.

In [ ]:
#Import mode function:
from scipy.stats import mode

#Determing the mode for each
outlet_size_mode = data.pivot_table(values='Outlet_Size', columns='Outlet_Type',aggfunc=(lambda x:mode(x.astype('str')).mode[0]))
print ('Mode for each Outlet_Type:')
print (outlet_size_mode)

#Get a boolean variable specifying missing Item_Weight values
missing_values = data['Outlet_Size'].isnull() 

#Impute data and check #missing values before and after imputation to confirm
print ('\nOrignal #missing: %d'% sum(missing_values))
data.loc[missing_values,'Outlet_Size'] = data.loc[missing_values,'Outlet_Type'].apply(lambda x: outlet_size_mode[x])
print (sum(data['Outlet_Size'].isnull()))

**Modify Item_Visibility**

We noticed that the minimum value here is 0, which makes no practical sense. Lets consider it like missing information and impute it with mean visibility of that product.

In [ ]:
#Determine average visibility of a product
visibility_avg = data.pivot_table(values='Item_Visibility', index='Item_Identifier')

#Impute 0 values with mean visibility of that product:
missing_values = (data['Item_Visibility'] == 0)

print ('Number of 0 values initially: %d'%sum(missing_values))
data.loc[missing_values,'Item_Visibility'] = data.loc[missing_values,'Item_Identifier'].apply(lambda x: visibility_avg.at[x, 'Item_Visibility'])
print ('Number of 0 values after modification: %d'%sum(data['Item_Visibility'] == 0))

**Create a broad category of Type of Item**

Earlier we saw that the Item_Type variable has 16 categories which might prove to be very useful in analysis. So its a good idea to combine them. One way could be to manually assign a new category to each. But there’s a catch here. If you look at the Item_Identifier, i.e. the unique ID of each item, it starts with either FD, DR or NC. If you see the categories, these look like being Food, Drinks and Non-Consumables. So I’ve used the Item_Identifier variable to create a new column:

In [ ]:
#Get the first two characters of ID:
data['Item_Type_Combined'] = data['Item_Identifier'].apply(lambda x: x[0:2])
#Rename them to more intuitive categories:
data['Item_Type_Combined'] = data['Item_Type_Combined'].map({'FD':'Food',
                                                             'NC':'Non-Consumable',
                                                             'DR':'Drinks'})
data['Item_Type_Combined'].value_counts()

**Modify categories of Item_Fat_Content**

We found typos and difference in representation in categories of Item_Fat_Content variable. This can be corrected as:

In [ ]:
#Change categories of low fat:
print('Original Categories:')
print(data['Item_Fat_Content'].value_counts())

print('\nModified Categories:')
data['Item_Fat_Content'] = data['Item_Fat_Content'].replace({'LF':'Low Fat',
                                                             'reg':'Regular',
                                                             'low fat':'Low Fat'})
print(data['Item_Fat_Content'].value_counts())

In [ ]:
df = data.loc[:,['Item_Outlet_Sales']]
df.head(2)

**Determine the years of operation of a store**
We wanted to make a new column depicting the years of operation of a store. This can be done as:

In [ ]:
data['Outlet_Years'] = 2009 - data['Outlet_Establishment_Year']
data['Outlet_Years'].describe()

In [ ]:
data.index = data['Outlet_Establishment_Year']
df = data.loc[:,['Item_Outlet_Sales']]
ts = df['Item_Outlet_Sales']

In [ ]:
temp_data = data.loc[data['Outlet_Establishment_Year'] == 1998]

In [ ]:
temp_data['Outlet_Type'].value_counts()

In [ ]:
test_temp_data = test.loc[test['Outlet_Establishment_Year'] == 1998]
test_temp_data['Outlet_Type'].value_counts()

**Numerical and One-Hot Coding of Categorical variables**

In [ ]:
#Import library:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#New variable for outlet
data['Outlet'] = le.fit_transform(data['Outlet_Identifier'])
var_mod = ['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Item_Type_Combined','Outlet_Type','Outlet','Item_Identifier']
le = LabelEncoder()
for i in var_mod:
    data[i] = le.fit_transform(data[i])

In [ ]:
#One Hot Coding:
data = pd.get_dummies(data, columns=['Item_Fat_Content','Outlet_Location_Type','Outlet_Size','Outlet_Type',
                              'Item_Type_Combined','Outlet'])

In [ ]:
#Drop the columns which have been converted to different types:
data.drop(['Item_Type','Outlet_Establishment_Year'],axis=1,inplace=True)
# data.to_csv("big_mart_sales.csv")
# #Divide into test and train:
train = data.loc[data['source']=="train"]
test = data.loc[data['source']=="test"]

# #Drop unnecessary columns:
test.drop(['source'],axis=1,inplace=True)
train.drop(['source','Item_Identifier','Outlet_Identifier'],axis=1,inplace=True)

In [ ]:
from sklearn.metrics import r2_score,mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

### Feature Scaling

In [ ]:
ss = MinMaxScaler()
for col in train.columns:
    train[col] = ss.fit_transform(train[[col]])
train.describe()

In [ ]:
plt.figure(figsize=(16,12))
ax = sns.heatmap(train.corr(), square=True, annot=True, fmt='.1f', linecolor='white',cmap='viridis')
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_yticklabels(ax.get_yticklabels(), rotation=30)           
plt.show()

### Observations :
>> #### The above correlation matrix shows that Item_Outlet_Sales is highly dependent on Item_MRP with correlation coefficient of 0.6

In [ ]:
y = train.Item_Outlet_Sales
X = train.drop('Item_Outlet_Sales',axis=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1,shuffle=True)

### Multiple Linear Regression

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)

### Evaluation Metrics

In [ ]:
print("Linear Regression Score : ",lr.score(X_test,y_test))
print("R2 Score : ",r2_score(y_test,y_pred))
print("Mean Squared Error : ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error : ",np.sqrt(mean_squared_error(y_test,y_pred)))


In [ ]:
print(lr.intercept_)
print(lr.coef_)

### Simple Linear Regression

In [ ]:
df = train['Item_MRP']
y = train.Item_Outlet_Sales
X = pd.DataFrame(df,columns = ['Item_MRP'])
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=1,shuffle=True)
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
y_pred = lr.predict(X_test)

### Evaluation Metrics

In [ ]:
print("Linear Regression Score : ",lr.score(X_test,y_test))
print("R2 Score : ",r2_score(y_test,y_pred))
print("Mean Squared Error : ",mean_squared_error(y_test,y_pred))
print("Root Mean Squared Error : ",np.sqrt(mean_squared_error(y_test,y_pred)))

### Conclusion :
>> #### The Multiple Linear Regression has average performance in case of multivariate data
>> #### The Simple Linear Regression on one attribute has very low score of 0.32 